In [23]:
data_root = 'drumData/'
sr = 48000 # this is the samplerate initially used to load the samples
total_limit = 100 #None # set this to 100 to export 100 samples
length_limit = sr/4 # set this to sr/4 to only export 250ms of audio per sample
downsampling_factor = 4
drumSprites = {}
limit = None

In [62]:
from os.path import join
from utils import ffmpeg_save_audio
import numpy as np
from multiprocessing import Pool
import pickle
import time as timeMod
import scipy.spatial
import scipy.spatial.distance as dist 
#%time samples = np.load(join(data_root, 'samples.npy'))

# Load samples

In [54]:
drumNames = pickle.load(open(data_root+'drumNames.pickle'))
drumLengths = pickle.load(open(data_root+'drumLengths.pickle'))
drumFingerPrints = {}
drumSamples = {}
for d in drumNames:
    %time drumSamples[d] = np.load(join(data_root, d+'_samples.npy'))

CPU times: user 6.34 ms, sys: 207 ms, total: 214 ms
Wall time: 303 ms
CPU times: user 682 µs, sys: 15.9 ms, total: 16.6 ms
Wall time: 25.9 ms
CPU times: user 4.14 ms, sys: 102 ms, total: 106 ms
Wall time: 167 ms
CPU times: user 3.06 ms, sys: 58.3 ms, total: 61.3 ms
Wall time: 121 ms
CPU times: user 351 µs, sys: 7.31 ms, total: 7.66 ms
Wall time: 15.7 ms
CPU times: user 2.74 ms, sys: 16.6 ms, total: 19.3 ms
Wall time: 60.9 ms
CPU times: user 1.74 ms, sys: 32.2 ms, total: 33.9 ms
Wall time: 66.4 ms


In [55]:
print drumNames, drumLengths

['kick', 'tom', 'snare', 'clap', 'hi.hat', 'ride', 'crash'] [5158, 422, 2546, 1324, 159, 228, 723]


# Compute sprites

In [56]:
# take a sample, crop it, and down sample it
def job(samp, length_limit, downsampling_factor):
    shortSamp = samp[:length_limit]
    if len(shortSamp) < length_limit:
        shortSamp = np.pad(shortSamp, (0, length_limit-len(shortSamp)), 'constant', constant_values=0)
    downShortSamp = np.interp(np.arange(0, length_limit, downsampling_factor), np.arange(0, length_limit), shortSamp)
    return downShortSamp

def calculateSprites(length_limit, downsampling_factor):
    _drumSprites = {}
    for d in drumNames:
        y = drumSamples[d]
        sprites = map(lambda samp: job(samp, length_limit, downsampling_factor), drumSamples[d][:limit])
        sprites = np.asarray(sprites).astype(np.float32)
        _drumSprites[d] = sprites
        print "generated spritesheet print for", d, sprites.shape
    return drumSprites

drumSprites = calculateSprites(length_limit, downsampling_factor)
data = np.concatenate([drumSprites[d] for d in drumNames])

generated spritesheet print for kick (5158, 3000)
generated spritesheet print for tom (422, 3000)
generated spritesheet print for snare (2546, 3000)
generated spritesheet print for clap (1324, 3000)
generated spritesheet print for hi.hat (159, 3000)
generated spritesheet print for ride (228, 3000)
generated spritesheet print for crash (723, 3000)


### Common setup 

In [57]:
# create point -> class hashmap
def getClassesPerSample(data):
    drumClasses = {}
    classIndex = 0
    for i in range(len(data)):
        if sum(drumLengths[0:classIndex+1]) <= i:
            classIndex += 1
        drumClasses[tuple(data[i])] = drumNames[classIndex]
    return drumClasses

drumClasses = getClassesPerSample(data)

numNeighbors = 10

## Arguments
## data -    an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i.
# numPerClass - Assumes that points from the same class are contiguous and in
#            the same order as drumNames. Should use drumLengths for this parameter
# calcFunc - The summary statistic you want to calcuate per class (mean, medain, etc)
def calculateFuncPerClass(data, numPerClass, calcFunc):
    segments = [0]+numPerClass
    for i in range(1, len(segments)):
        segments[i] = segments[i] + segments[i-1]
    valuesPerClass = []
    for i in range(len(segments)-1):
        valuesPerClass.append(calcFunc(data[segments[i]:segments[i+1]]))
    return valuesPerClass

### pairwise distance implementation

In [63]:
#using https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.spatial.distance.cdist.html
#and argpartition from - https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array

## Arguments - pointData is m x n np array, with n points of m dimensions
# pointD numNeighbors is the number of nearest neighbors for which to compare classes
#
## returns - an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i
def calculateKNNClassAccuracy_pairwise(pointData, numNeighbors, printTimes=False):
    startTime = timeMod.time()
    pairwiseDist = dist.cdist(pointData, pointData)
    if printTimes:
        print "pairwise distances calculated", timeMod.time() - startTime
    kPartition = np.argpartition(pairwiseDist, -numNeighbors)
    if printTimes:
        print "partitions calculated", timeMod.time() - startTime
    fracSameNeighborClasses = np.zeros((len(data)))
    
    for i in range(len(pairwiseDist)):
        neighborIndexes = kPartition[i][-numNeighbors:]
        neighbors = [data[ind] for ind in neighborIndexes]
        
        sampleClass = drumClasses[tuple(data[i])]
        neighborClasses = [drumClasses[tuple(neighbor)] for neighbor in neighbors]
        numSameNeighborClasses = len(filter(lambda c : c == sampleClass, neighborClasses))
        fracSameNeighborClasses[i] = numSameNeighborClasses * 1.0 / numNeighbors
    
    if printTimes:
        print "knn classes calculated", timeMod.time() - startTime
        
    return fracSameNeighborClasses

#classAccuracies_LD = calculateKNNClassAccuracy(newData, numNeighbors, True)
classAccuracies_HD = calculateKNNClassAccuracy_pairwise(data, numNeighbors, True)

pairwise distances calculated 333.950196028
partitions calculated 335.279674053
knn classes calculated 360.15080595
